In [1]:
import pandas as pd
import os
import gc
import lightgbm as lgb
from sklearn.linear_model import SGDRegressor, LinearRegression, Ridge
from sklearn.preprocessing import MinMaxScaler
from gensim.models import Word2Vec
import math
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold, KFold, GroupKFold
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, log_loss
import matplotlib.pyplot as plt
import time
import warnings
warnings.filterwarnings('ignore')

# 数据读取

In [2]:
disease_feature1 = pd.read_csv("/home/mw/input/data8766/训练集/训练集/disease_feature1.csv")
disease_feature2 = pd.read_csv("/home/mw/input/data8766/训练集/训练集/disease_feature2.csv")
disease_feature3 = pd.read_csv("/home/mw/input/data8766/训练集/训练集/disease_feature3.csv")

train_answer = pd.read_csv("/home/mw/input/data8766/训练集/训练集/train_answer.csv")
train_food = pd.read_csv("/home/mw/input/data8766/训练集/训练集/train_food.csv")

# preliminary_a_food = pd.read_csv("/home/mw/input/data8068/preliminary_b_food.csv")
# preliminary_a_submit_sample = pd.read_csv("/home/mw/input/data8068/preliminary_b_submit_sample.csv")

In [3]:
semi_train_answer = pd.read_csv("/home/mw/input/data6690/semi_train_answer.csv")
semi_train_answer
#   food_feature = pd.read_csv(semi_food_path)[['food_id']].assign(key = 1)

,food_id,disease_id,related_score
0,food_0,disease_208,0
1,food_0,disease_1166,0
2,food_0,disease_1418,0
3,food_0,disease_76,0
4,food_0,disease_579,0
5,food_0,disease_760,0
6,food_0,disease_1425,0
7,food_0,disease_767,0
8,food_0,disease_1005,0
9,food_0,disease_671,0


In [110]:
len(preliminary_a_submit_sample)

47212

In [111]:
del preliminary_a_submit_sample['related_prob']

In [112]:
data = pd.concat([train_answer, preliminary_a_submit_sample], axis = 0).reset_index(drop=True)
data.head()

,disease_id,food_id,related
0,disease_998,food_0,0.0
1,disease_861,food_0,0.0
2,disease_559,food_0,0.0
3,disease_841,food_0,0.0
4,disease_81,food_0,0.0


这里直接使用每个变量后的数字进行编码，当然也可以使用labelencoder的方式。

In [113]:
data['food'] = data['food_id'].apply(lambda x : int(x.split('_')[1]))
data['disease'] = data['disease_id'].apply(lambda x : int(x.split('_')[1]))

In [114]:
food = pd.concat([train_food, preliminary_a_food], axis = 0).reset_index(drop=True)
food.head()

,food_id,N_0,N_1,N_2,N_3,N_4,N_5,N_6,N_7,N_8,...,N_202,N_203,N_204,N_205,N_206,N_207,N_208,N_209,N_210,N_211
0,food_0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,0.02,0.0,NaN,NaN,30.5,92.82,NaN,0.92
1,food_1,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,23.90,0.0,NaN,NaN,0.0,2.41,NaN,3.31
2,food_4,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,0.12,0.0,NaN,NaN,3.5,15.46,NaN,0.36
3,food_5,NaN,NaN,NaN,0.068,0.0,0.045,0.75,0.314,NaN,...,NaN,NaN,0.89,0.0,NaN,NaN,3.3,86.35,NaN,0.20
4,food_6,NaN,NaN,NaN,0.115,0.0,0.091,0.58,0.508,NaN,...,NaN,NaN,1.13,0.0,0.0,NaN,41.6,93.22,NaN,0.54


# 目标编码                      
                      
由于本题只有两个离散变量food_id和disease_id，而测试集中都是新的foodid。

In [115]:
cat_list = ['disease']
def stat(df, df_merge, group_by, agg):
    group = df.groupby(group_by).agg(agg)

    columns = []
    for on, methods in agg.items():
        for method in methods:
            columns.append('{}_{}_{}'.format('_'.join(group_by), on, method))
    group.columns = columns
    group.reset_index(inplace=True)
    df_merge = df_merge.merge(group, on=group_by, how='left')

    del (group)
    gc.collect()
    return df_merge


def statis_feat(df_know, df_unknow,cat_list):
    for f in tqdm(cat_list):
        df_unknow = stat(df_know, df_unknow, [f], {'related': ['mean','std']})

    return df_unknow


df_train = data[~data['related'].isnull()]
df_train = df_train.reset_index(drop=True)
df_test = data[data['related'].isnull()]

df_stas_feat = None
kf = StratifiedKFold(n_splits=5, random_state=2020, shuffle=True)
for train_index, val_index in kf.split(df_train, df_train['related']):
    df_fold_train = df_train.iloc[train_index]
    df_fold_val = df_train.iloc[val_index]

    df_fold_val = statis_feat(df_fold_train, df_fold_val,cat_list)
    df_stas_feat = pd.concat([df_stas_feat, df_fold_val], axis=0)

    del (df_fold_train)
    del (df_fold_val)
    gc.collect()

df_test = statis_feat(df_train, df_test,cat_list)
data = pd.concat([df_stas_feat, df_test], axis=0)
data = data.reset_index(drop=True)

del (df_stas_feat)
del (df_train)
del (df_test)

100%|██████████| 1/1 [00:00<00:00, 12.21it/s]


In [116]:
data.head()

,disease_id,food_id,related,food,disease,disease_related_mean,disease_related_std
0,disease_201,food_0,0.0,0,201,0.246324,0.431663
1,disease_1191,food_0,0.0,0,1191,0.003650,0.060412
2,disease_629,food_0,0.0,0,629,0.097826,0.297619
3,disease_1026,food_0,0.0,0,1026,0.007380,0.085748
4,disease_1236,food_0,0.0,0,1236,0.064057,0.245291


# 疾病特征处理                      
                      
这里我们使用TruncatedSVD的方法，对疾病特征进行降维，维度均为128。

In [117]:
# f_col = [col for col in disease_feature1.columns if 'F' in col]

In [118]:
from sklearn.decomposition import PCA,KernelPCA,TruncatedSVD,SparsePCA
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,MinMaxScaler
SEED = 7
def pca_fea(data, feats, n_components=10, name='tsvd', load=False):

    tsvd = Pipeline([
        ('std', MinMaxScaler()),
        # ('tsvd', TruncatedSVD(n_components=n_components, n_iter=20, random_state=SEED)),
        # ('pca', SparsePCA(n_components=n_components, max_iter=200, random_state=SEED,alpha=0.1,n_jobs=8))
        ('pca', PCA(n_components=n_components, random_state=SEED))

    ])
    tsvd.fit(data[feats])
    data_id = data['disease_id']
    deal_data = pd.DataFrame(tsvd.transform(data[feats]), columns=[f'{name}_{i}' for i in range(n_components)])
    deal_data.insert(0, 'disease_id', data['disease_id'])
    return deal_data

In [119]:
n_disease_tsvd1 = 125
feat1 = pca_fea(
    disease_feature1, 
    [item for item in disease_feature1.columns if item not in ['disease_id']], 
    n_components=n_disease_tsvd1, 
    name='disease1_pca'
)
n_disease_tsvd2 = 220
feat2 = pca_fea(
    disease_feature2, 
    [item for item in disease_feature2.columns if item not in ['disease_id']], 
    n_components=n_disease_tsvd2, 
    name='disease2_pca'
)
n_disease_tsvd3 = 130
feat3 = pca_fea(
    disease_feature3, 
    [item for item in disease_feature3.columns if item not in ['disease_id']], 
    n_components=n_disease_tsvd3, 
    name='disease3_pca'
)

In [120]:
# disease_feature_1_ = disease_feature1.copy()
# from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,TfidfTransformer 
# from sklearn.decomposition import TruncatedSVD, SparsePCA,PCA
# disease_feature_1_ = disease_feature_1_.fillna(0)
# decom=TruncatedSVD(n_components=128, n_iter = 20, random_state=2023) 

# decom_x=decom.fit_transform(disease_feature_1_.iloc[:,1:]) 
# decom_feas=pd.DataFrame(decom_x)
# decom_feas.columns=['disease1_svd_'+str(i) for i in range(decom_x.shape[1])]

In [121]:
# disease_feature1 = disease_feature1[['disease_id']]
# for col in decom_feas:
#     disease_feature1[col] = decom_feas[col]

In [122]:
# f_col = [col for col in disease_feature2.columns if 'F' in col]

In [123]:
# disease_feature_2_ = disease_feature2.copy()
# from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,TfidfTransformer 
# from sklearn.decomposition import TruncatedSVD, SparsePCA
# disease_feature_2_ = disease_feature_2_.fillna(0)
# decom=TruncatedSVD(n_components=128, n_iter = 20, random_state=2023) 

# decom_x=decom.fit_transform(disease_feature_2_.iloc[:,1:]) 
# decom_feas=pd.DataFrame(decom_x)
# decom_feas.columns=['disease2_svd_'+str(i) for i in range(decom_x.shape[1])]

In [124]:
# disease_feature2 = disease_feature2[['disease_id']]
# for col in decom_feas:
#     disease_feature2[col] = decom_feas[col]

In [125]:
# f_col = [col for col in disease_feature3.columns if 'F' in col]

In [126]:
# disease_feature_3_ = disease_feature3.copy()
# from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,TfidfTransformer 
# from sklearn.decomposition import TruncatedSVD, SparsePCA
# disease_feature_3_ = disease_feature_3_.fillna(0)
# decom=TruncatedSVD(n_components=128, n_iter = 20, random_state=2023) 

# decom_x=decom.fit_transform(disease_feature_3_.iloc[:,1:]) 
# decom_feas=pd.DataFrame(decom_x)
# decom_feas.columns=['disease3_svd_'+str(i) for i in range(decom_x.shape[1])]

In [127]:
# disease_feature3 = disease_feature3[['disease_id']]
# for col in decom_feas:
#     disease_feature3[col] = decom_feas[col]

In [128]:
drop_rate = 0.1
per_mis = list(food.isnull().sum() / len(food))
df_missing = pd.DataFrame({'列名': food.columns,'缺失率': per_mis})
df_missing.sort_values('缺失率', inplace=True,ascending=False)
miss_fea = list(df_missing[df_missing['缺失率']<=drop_rate]['列名'])
print(len(miss_fea))
miss_fea

18


['N_198',
 'N_33',
 'N_211',
 'N_82',
 'N_101',
 'N_111',
 'N_42',
 'N_177',
 'N_165',
 'N_146',
 'N_113',
 'N_17',
 'N_106',
 'N_14',
 'N_74',
 'N_209',
 'N_188',
 'food_id']

In [129]:
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler,MinMaxScaler
# d_c = []
df_food = food[miss_fea]
df_food = df_food.drop(columns='food_id')
# for f in df_food.columns:
#     if df_food[f].nunique() >2:
#         d_c.append(f)
df_food = df_food.fillna(-1)

df_dis_1 = feat1.drop(columns='disease_id')
df_dis_2 = feat2.drop(columns='disease_id')
df_dis_3 = feat3.drop(columns='disease_id')

def k_s(df,a,b):
    K_list = []
    score_list = []

    for n in range(a, b):
        gmm = GaussianMixture(n_components=n, covariance_type='full',random_state= 7)
        gmm.fit(df)
        labels = gmm.fit_predict(df)
        K_list.append(n)
        for i in range(df.shape[0]):
            s = silhouette_score(df, labels, metric='euclidean')
        score_list.append(s)
    ind_s = score_list.index(max(score_list))
    
    return K_list[ind_s],max(score_list)

print('搜索疾病特征第一类的K值')
k1,s1 = k_s(df_dis_1,2,4)
print('K值为：{}'.format(k1),'最大轮廓系数为：{}'.format(s1))
print('搜索疾病特征第二类的K值')
k2,s2 = k_s(df_dis_2,2,4)
print('K值为：{}'.format(k2),'最大轮廓系数为：{}'.format(s2))
print('搜索疾病特征第三类的K值')
k3,s3 = k_s(df_dis_3,52,58)
print('K值为：{}'.format(k3),'最大轮廓系数为：{}'.format(s3))
print('搜索食物分类的K值')
scaler = StandardScaler()
df_food_st = scaler.fit_transform(df_food)
k4,s4 = k_s(df_food_st,2,15)
print('K值为：{}'.format(k4),'最大轮廓系数为：{}'.format(s4))


搜索疾病特征第一类的K值
K值为：2 最大轮廓系数为：0.6728047237088757
搜索疾病特征第二类的K值
K值为：2 最大轮廓系数为：0.670575887290354
搜索疾病特征第三类的K值
K值为：55 最大轮廓系数为：0.2668225302211355
搜索食物分类的K值
K值为：6 最大轮廓系数为：0.26710863482580627


In [130]:
def dis_clu(X,n):
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    gmm = GaussianMixture(n_components=n, covariance_type='full')
    gmm.fit(X)
    labels = gmm.fit_predict(X)
    return labels
clu1 = dis_clu(df_dis_1,k1)
feat1['clu1'] = clu1
clu2 = dis_clu(df_dis_2,k2)
feat2['clu2'] = clu2
clu3 = dis_clu(df_dis_3,k3)
feat3['clu3'] = clu3
food_clu = dis_clu(df_food_st,k4)
food['food_clu'] = food_clu

In [131]:
food['food_clu'].mean()

1.7866379310344827

In [132]:
data = data.merge(food, on = 'food_id', how = 'left')
data = data.merge(feat1, on = 'disease_id', how = 'left')
data = data.merge(feat2, on = 'disease_id', how = 'left')
data = data.merge(feat3, on = 'disease_id', how = 'left')
data.head()

,disease_id,food_id,related,food,disease,disease_related_mean,disease_related_std,N_0,N_1,N_2,...,disease3_tsvd_121,disease3_tsvd_122,disease3_tsvd_123,disease3_tsvd_124,disease3_tsvd_125,disease3_tsvd_126,disease3_tsvd_127,disease3_tsvd_128,disease3_tsvd_129,clu3
0,disease_201,food_0,0.0,0,201,0.246324,0.431663,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,disease_1191,food_0,0.0,0,1191,0.003650,0.060412,NaN,NaN,NaN,...,0.144597,-0.026062,-0.069724,-0.050384,0.015302,-0.019582,-0.009329,-0.012747,0.036910,21.0
2,disease_629,food_0,0.0,0,629,0.097826,0.297619,NaN,NaN,NaN,...,0.009950,-0.093899,0.015182,0.062358,0.027275,-0.025317,-0.013170,-0.007581,0.037341,18.0
3,disease_1026,food_0,0.0,0,1026,0.007380,0.085748,NaN,NaN,NaN,...,-0.107328,0.061280,0.166828,-0.056331,0.173042,0.236797,0.065835,-0.039358,0.194265,5.0
4,disease_1236,food_0,0.0,0,1236,0.064057,0.245291,NaN,NaN,NaN,...,0.087648,0.020780,0.062447,0.134004,-0.151617,0.039114,0.057732,0.050860,0.069073,36.0


# 交叉特征                      
                      
这里我们按照特征重要性选取靠前的部分特征进行交叉。

In [147]:
topn = ['N_33', 'N_198', 'N_165','N_74','food','N_17','N_165','N_147','N_42','N_146']
for i in range(len(topn)):
    for j in range(i + 1, len(topn)):
        data[f'{topn[i]}+{topn[j]}'] = data[topn[i]] + data[topn[j]]
        data[f'{topn[i]}-{topn[j]}'] = data[topn[i]] - data[topn[j]]
        data[f'{topn[i]}*{topn[j]}'] = data[topn[i]] * data[topn[j]]
        data[f'{topn[i]}/{topn[j]}'] = data[topn[i]] / (data[topn[j]]+1e-5)

In [148]:
drop_cols = ['disease_id', 'food_id', 'related']

# 特征筛选                      
                      
去除掉只有单一取值的特征

In [149]:
for f in data.columns:
    if data[f].nunique() < 2:
        drop_cols.append(f)

In [150]:
test_df = data[data["related"].isnull() == True].copy().reset_index(drop=True)
train_df = data[~data["related"].isnull() == True].copy().reset_index(drop=True)

In [151]:
test_df['food_clu'].nunique()

6

In [152]:
test_df['food_clu'].mean()

1.6551724137931034

In [153]:
feature_name = [f for f in train_df.columns if f not in drop_cols]
X_train = train_df[feature_name].reset_index(drop=True)
X_test = test_df[feature_name].reset_index(drop=True)
y = train_df['related'].reset_index(drop=True)
print(len(feature_name))
print(feature_name)

822
['food', 'disease', 'disease_related_mean', 'disease_related_std', 'N_0', 'N_1', 'N_3', 'N_4', 'N_5', 'N_6', 'N_7', 'N_9', 'N_10', 'N_11', 'N_13', 'N_14', 'N_16', 'N_17', 'N_18', 'N_19', 'N_20', 'N_22', 'N_28', 'N_29', 'N_30', 'N_31', 'N_33', 'N_35', 'N_37', 'N_40', 'N_42', 'N_43', 'N_44', 'N_45', 'N_46', 'N_47', 'N_48', 'N_49', 'N_50', 'N_51', 'N_52', 'N_53', 'N_54', 'N_55', 'N_56', 'N_57', 'N_58', 'N_59', 'N_60', 'N_61', 'N_62', 'N_63', 'N_64', 'N_67', 'N_68', 'N_69', 'N_71', 'N_72', 'N_73', 'N_74', 'N_75', 'N_76', 'N_77', 'N_78', 'N_79', 'N_80', 'N_81', 'N_82', 'N_84', 'N_85', 'N_86', 'N_87', 'N_88', 'N_89', 'N_90', 'N_91', 'N_92', 'N_93', 'N_94', 'N_95', 'N_96', 'N_97', 'N_98', 'N_99', 'N_100', 'N_101', 'N_102', 'N_104', 'N_105', 'N_106', 'N_109', 'N_111', 'N_112', 'N_113', 'N_114', 'N_115', 'N_116', 'N_118', 'N_119', 'N_120', 'N_121', 'N_122', 'N_123', 'N_124', 'N_125', 'N_126', 'N_127', 'N_128', 'N_129', 'N_131', 'N_132', 'N_133', 'N_134', 'N_135', 'N_136', 'N_138', 'N_139', 

In [169]:
print(test_df.shape)

(47212, 858)


# 模型训练                      
                      
本次仅使用lightgbm模型来训练。

In [170]:
train_pred = {}
test_pred = {}

In [171]:
seeds = [222222222]
num_model_seed = 1
oof = np.zeros(X_train.shape[0])
prediction = np.zeros(X_test.shape[0])
feat_imp_df = pd.DataFrame({'feats': feature_name, 'imp': 0})
parameters = {
    'learning_rate': 0.01,
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 64,
    'lambda_l1': 0.5,
    'lambda_l2': 0.5,
    'feature_fraction': 0.7,
    'bagging_fraction': 0.7,
    'bagging_freq': 5,
    'seed': 222222,
    'bagging_seed': 222222,
    'feature_fraction_seed': 222222,
    'min_data_in_leaf': 20,
    'verbose': -1, 
    'n_jobs':8
}
fold = 5
for model_seed in range(num_model_seed):
    print(seeds[model_seed],"--------------------------------------------------------------------------------------------")
    oof_cat = np.zeros(X_train.shape[0])
    prediction_cat = np.zeros(X_test.shape[0])
    skf = StratifiedKFold(n_splits=fold, random_state=seeds[model_seed], shuffle=True)
    for index, (train_index, test_index) in enumerate(skf.split(X_train, y)):
        train_x, test_x, train_y, test_y = X_train[feature_name].iloc[train_index], X_train[feature_name].iloc[test_index], y.iloc[train_index], y.iloc[test_index]
        dtrain = lgb.Dataset(train_x, label=train_y)
        dval = lgb.Dataset(test_x, label=test_y)
        lgb_model = lgb.train(
            parameters,
            dtrain,
            num_boost_round=10000,
            valid_sets=[dval],
            early_stopping_rounds=100,
            verbose_eval=100, )
        oof_cat[test_index] += lgb_model.predict(test_x,num_iteration=lgb_model.best_iteration)
        prediction_cat += lgb_model.predict(X_test,num_iteration=lgb_model.best_iteration) / fold
        feat_imp_df['imp'] += lgb_model.feature_importance()

        del train_x
        del test_x
        del train_y
        del test_y
        del lgb_model
    oof += oof_cat / num_model_seed
    prediction += prediction_cat / num_model_seed
gc.collect()

222222222 --------------------------------------------------------------------------------------------
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.954125
[200]	valid_0's auc: 0.962575
[300]	valid_0's auc: 0.967144
[400]	valid_0's auc: 0.970241
[500]	valid_0's auc: 0.972097
[600]	valid_0's auc: 0.97366
[700]	valid_0's auc: 0.97461
[800]	valid_0's auc: 0.975444
[900]	valid_0's auc: 0.976168
[1000]	valid_0's auc: 0.976794
[1100]	valid_0's auc: 0.977279
[1200]	valid_0's auc: 0.977706
[1300]	valid_0's auc: 0.97808
[1400]	valid_0's auc: 0.978412
[1500]	valid_0's auc: 0.978665
[1600]	valid_0's auc: 0.978881
[1700]	valid_0's auc: 0.979112
[1800]	valid_0's auc: 0.979324
[1900]	valid_0's auc: 0.979542
[2000]	valid_0's auc: 0.979761
[2100]	valid_0's auc: 0.979932
[2200]	valid_0's auc: 0.980057
[2300]	valid_0's auc: 0.980204
[2400]	valid_0's auc: 0.980368
[2500]	valid_0's auc: 0.980485
[2600]	valid_0's auc: 0.980555
[2700]	valid_0's auc: 0.980632
[2800]	va

6168

In [172]:
feat_imp_df.sort_values(by=['imp'],axis=0,ascending=False,inplace=True)
feat_imp_df.head(100)

,feats,imp
2,disease_related_mean,27751
3,disease_related_std,13442
165,N_193,10541
160,N_188,10480
20,N_20,10380
47,N_59,10032
9,N_6,9094
49,N_61,8940
773,N_74/N_146,8565
98,N_119,8498


In [173]:
feat_imp_df[feat_imp_df['feats'].str.contains('dis')]

,feats,imp
2,disease_related_mean,27751
3,disease_related_std,13442
549,disease3_tsvd_17,5654
578,disease3_tsvd_46,4656
544,disease3_tsvd_12,4449
657,disease3_tsvd_125,4290
534,disease3_tsvd_2,4238
572,disease3_tsvd_40,4154
656,disease3_tsvd_124,4141
547,disease3_tsvd_15,4006


In [174]:
feat_imp_df[feat_imp_df['feats'] == 'food_clu']

,feats,imp
184,food_clu,459


In [175]:
train_pred['lgb'] = oof
test_pred['lgb'] = prediction

In [176]:
print("lgb train auc: ", roc_auc_score(y, train_pred['lgb']))

lgb train auc:  0.981635726110887


lgb train auc:  0.9809961140871605

In [177]:
scores = []; thresholds = []
best_score = 0; best_threshold = 0

for threshold in np.arange(0.1,0.9,0.01):
    print(f'{threshold:.02f}, ',end='')
    preds = (train_pred['lgb'].reshape((-1)) > threshold).astype('int')
    m = f1_score(y.values.reshape((-1)), preds, average='binary')   
    scores.append(m)
    thresholds.append(threshold)
    if m>best_score:
        best_score = m
        best_threshold = threshold

0.10, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.20, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.30, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.40, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.50, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.60, 0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69, 0.70, 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.80, 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89, 

In [178]:
import matplotlib.pyplot as plt

# PLOT THRESHOLD VS. F1_SCORE
plt.figure(figsize=(20,5))
plt.plot(thresholds,scores,'-o',color='blue')
plt.scatter([best_threshold], [best_score], color='blue', s=300, alpha=1)
plt.xlabel('Threshold',size=14)
plt.ylabel('Validation F1 Score',size=14)
plt.title(f'Threshold vs. F1_Score with Best F1_Score = {best_score:.3f} at Best Threshold = {best_threshold:.3}',size=18)
plt.show()

<Figure size 1440x360 with 1 Axes>

In [179]:
auc = roc_auc_score(y, train_pred['lgb'])
f1 = best_score
print((auc + f1) / 2)

0.9058014320839853


In [161]:
0.9022519866225931

0.9022519866225931

# 生成提交结果                      
                      
这里我们控制1的个数为4100个左右，最终线上得分0.7956687074375137，f1:0.6540415704387991, auc:0.9372958444362285。

In [180]:
# label=[1 if x >= 0.265+0.235 else 0 for x in prediction+0.235]
# np.sum(label)

label=[1 if x >= 0.26+0.24 else 0 for x in prediction]
np.sum(label)

2870

In [181]:
prediction

array([2.13893181e-04, 7.57007651e-04, 7.32585320e-01, ...,
       5.36888482e-05, 5.97403718e-05, 1.88192546e-02])

In [70]:
preliminary_a_submit_sample['related_prob'] = prediction+0.24

In [71]:
preliminary_a_submit_sample.to_csv('submit152.csv', index=False)

In [98]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

train_feat =  oof
test_feat =  prediction+0.24

sns.kdeplot(train_feat, shade = True, color='r', label = 'train')
sns.kdeplot(test_feat, shade = True, color='b', label = 'test')
plt.xlabel('Feature')
plt.legend()
plt.show()

<Figure size 432x288 with 1 Axes>

# 进一步优化                      
                      
1. 由于foodid只有训练集有，那么是否可以使用food侧的特征，做相似度模型，例如共现矩阵、tfidf、embedding等。                      
                      
2. 目标编码做了之后，线下会涨很多但是线上长得比较少，还是过拟合比较严重，是否可以考虑根据疾病特征做聚类，然后减轻这种情况。同理food侧特征也可以做聚类，用来解决测试集都是训练集未曾出现过的id的问题。                      
                      
3. 交叉特征里面，对于food侧只取了一部分，是否可以多取一点（进一步上述base可以通过筛选food特征提升至7976的分数，但是会很抖）。                      
                      
4. 特征筛选的地方，并没有剔除缺失率高的特征，也没有根据对抗验证进行筛选，或许可以进行尝试。                      
                      
5. 模型训练的参数，学习率太低，叶子节点数太高，导致模型过拟合比较严重，可以考虑调参（可以上分）。                      
                      
6. 目前仅使用了lightgbm模型，可以考虑xgboost，catboost模型，进行模型的集成（xgb貌似还不错，需要祖传参数）。                      
                      
7. 上述base是1的个数为4100个，可以调整不同的个数来测试，亲测不同的特征组合哪怕只是添加一个，最优的1的个数都是不同的。米哥开源的7949的代码，只需调整1的个数就可以7956，可以考虑将两份base结合看是否可以得到更高的分数。                      
                      
8. svd的维度是可以调整的，不一定每个都是一样的。以及还可以使用pca的方法，不过需要进行归一化。

In [182]:
def prob_post_processing(train_oof, test_pred, threshold):
    train_oof = 1/(1+np.exp((-train_oof + threshold)*3))
    test_pred = 1/(1+np.exp((-test_pred + threshold)*3))
    return train_oof, test_pred
def Find_Optimal_Cutoff_F1(y, prob, verbose=False):
    precision, recall, threshold = sklearn.metrics.precision_recall_curve(y,prob)
    y = 2*(precision*recall)/(precision+recall)
    Youden_index = np.argmax(y)
    optimal_threshold = threshold[Youden_index]
    if verbose: print("optimal_threshold", optimal_threshold)
    return optimal_threshold

In [183]:
import sklearn
optimal_threshold = prediction[prediction.argsort()][-4582]
print('test thres', optimal_threshold)
oof_o, test_pred_o = prob_post_processing(oof, prediction, optimal_threshold)
optimal_threshold = Find_Optimal_Cutoff_F1(y, oof, verbose=True)

test thres 0.20468469780759957
optimal_threshold 0.38855630976448624


In [184]:
y_pred = (oof >= optimal_threshold).astype(int)
y_true = y

auc_value = sklearn.metrics.roc_auc_score(y, oof)
f1_vlaue = sklearn.metrics.f1_score(y_true, y_pred)
p_value = sklearn.metrics.precision_score(y_true, y_pred)
r_value = sklearn.metrics.recall_score(y_true, y_pred)

print('total_score:', (auc_value+f1_vlaue)/2)
print("auc_score:", auc_value)
print("f1_score:", f1_vlaue)
print("precision_score(查准率):", p_value)
print("recall_score(查全率):", r_value)
score_str = f"{(auc_value+f1_vlaue)/2:.8f}_{auc_value:.5f}_{f1_vlaue:.5f}"

total_score: 0.9058947974859073
auc_score: 0.981635726110887
f1_score: 0.8301538688609276
precision_score(查准率): 0.8645096833691976
recall_score(查全率): 0.7984243026474555


In [185]:
sum(test_pred_o )/len(list(test_pred_o ))

0.397937554235608

In [186]:
label = [1 if x >= 0.5 else 0 for x in test_pred_o ]
sum(label)

4582

In [187]:
preliminary_a_submit_sample['related_prob'] = [x for x in test_pred_o]
preliminary_a_submit_sample.to_csv('sub_lgbm22.csv', index=False)

In [188]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

train_feat =  oof_o
test_feat =  test_pred_o

sns.kdeplot(train_feat, shade = True, color='r', label = 'train')
sns.kdeplot(test_feat, shade = True, color='b', label = 'test')
plt.xlabel('Feature')
plt.legend()
plt.show()

<Figure size 432x288 with 1 Axes>